In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
from pathlib import Path

In [4]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

In [5]:
update_path = Path('/app/data/raw/queues_2021_clean_data.xlsx')
assert update_path.exists()

In [6]:
iso = pd.read_excel(update_path, sheet_name='data', parse_dates=True)

In [7]:
iso.head(3)

,q_id,q_status,q_date,q_year,ia_date,wd_date,on_date,entity,project_name,developer,utility,county_1,county_2,county_3,state,region,service,poi_name,prop_date,prop_year,ia_status_raw,ia_status_clean,type_clean,type1,type2,type3,mw1,mw2,mw3
0,2,operational,2/3/2000,2000.0,NaN,NaN,1/6/2009,CAISO,GATEWAY GENERATING FACILITY (FKA CONTRA COSTA ...,NaN,PGAE,contra costa,NaN,NaN,CA,CAISO,NaN,Contra Costa Power Plant 230 kV bus,11/28/2007,2007.0,Executed,Operational,Gas,Gas,NaN,NaN,590.0,NaN,NaN
1,3,operational,6/14/2000,2000.0,NaN,NaN,6/1/2013,CAISO,CPV SENTINEL (FKA INTERGEN OCOTILLO),NaN,SCE,riverside,NaN,NaN,CA,CAISO,Full Capacity,Devers Substation 230kV Bus,1/1/2004,2004.0,Executed,Operational,Gas,Gas,NaN,NaN,850.0,NaN,NaN
2,4,operational,8/8/2000,2000.0,NaN,NaN,10/14/2005,CAISO,PALOMAR ENERGY PROJECT,0,SDGE,san diego,NaN,NaN,CA,CAISO,NaN,Palomar 230 kV,6/1/2001,2001.0,Executed,Operational,Gas,Gas,NaN,NaN,521.0,NaN,NaN


In [8]:
iso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24381 entries, 0 to 24380
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   q_id             24380 non-null  object 
 1   q_status         24381 non-null  object 
 2   q_date           24303 non-null  object 
 3   q_year           24296 non-null  float64
 4   ia_date          2526 non-null   object 
 5   wd_date          6323 non-null   object 
 6   on_date          1571 non-null   object 
 7   entity           24381 non-null  object 
 8   project_name     8609 non-null   object 
 9   developer        3567 non-null   object 
 10  utility          20999 non-null  object 
 11  county_1         23963 non-null  object 
 12  county_2         71 non-null     object 
 13  county_3         19 non-null     object 
 14  state            23973 non-null  object 
 15  region           24381 non-null  object 
 16  service          18920 non-null  object 
 17  poi_name    

In [9]:
iso.nunique()

q_id               19011
q_status               4
q_date              5979
q_year                27
ia_date             1609
wd_date             3103
on_date             1081
entity                42
project_name        7582
developer           1948
utility              228
county_1            1723
county_2              50
county_3              15
state                 50
region                 9
service               83
poi_name           17006
prop_date           3945
prop_year             45
ia_status_raw        862
ia_status_clean       11
type_clean            34
type1                 14
type2                  6
type3                  2
mw1                 1071
mw2                  271
mw3                    4
dtype: int64

In [10]:
iso.shape

(24381, 29)

In [11]:
iso['q_status'].value_counts()

withdrawn      12580
active          8133
operational     3439
suspended        229
Name: q_status, dtype: int64

In [12]:
active = iso.query('q_status == "active"')

In [13]:
active.shape

(8133, 29)

In [14]:
active['q_id'].nunique()

7573

In [15]:
active[['q_id', 'utility']].duplicated().sum()

64

In [16]:
from dbcp.extract.lbnl_iso_queue_2021 import extract

/app/.local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [17]:
# rename cols
iso = extract(update_path)['lbnl_iso_queue_2021']

In [18]:
iso.head(2)

,queue_id,queue_status,queue_date,queue_year,interconnection_date,date_withdrawn,date_operational,entity,project_name,developer,utility,county_1,county_2,county_3,state,region,interconnection_service_type,point_of_interconnection,date_proposed,year_proposed,interconnection_status_raw,interconnection_status_lbnl,resource_type_lbnl,resource_type_1,resource_type_2,resource_type_3,capacity_mw_resource_1,capacity_mw_resource_2,capacity_mw_resource_3
0,2,operational,2/3/2000,2000.0,NaN,NaN,1/6/2009,CAISO,GATEWAY GENERATING FACILITY (FKA CONTRA COSTA ...,NaN,PGAE,contra costa,NaN,NaN,CA,CAISO,NaN,Contra Costa Power Plant 230 kV bus,11/28/2007,2007.0,Executed,Operational,Gas,Gas,NaN,NaN,590.0,NaN,NaN
1,3,operational,6/14/2000,2000.0,NaN,NaN,6/1/2013,CAISO,CPV SENTINEL (FKA INTERGEN OCOTILLO),NaN,SCE,riverside,NaN,NaN,CA,CAISO,Full Capacity,Devers Substation 230kV Bus,1/1/2004,2004.0,Executed,Operational,Gas,Gas,NaN,NaN,850.0,NaN,NaN


In [20]:
active = iso.query('queue_status == "active"')

In [21]:
active.shape

(8133, 29)

In [22]:
cap_cols = [col for col in iso.columns if col.startswith('capacity_mw_resource_')]
cap_cols

['capacity_mw_resource_1', 'capacity_mw_resource_2', 'capacity_mw_resource_3']

In [23]:
# negative capacity still exists and still makes no sense (except maybe some repowering projects)
active[active[cap_cols].lt(0).any(axis=1)]

,queue_id,queue_status,queue_date,queue_year,interconnection_date,date_withdrawn,date_operational,entity,project_name,developer,utility,county_1,county_2,county_3,state,region,interconnection_service_type,point_of_interconnection,date_proposed,year_proposed,interconnection_status_raw,interconnection_status_lbnl,resource_type_lbnl,resource_type_1,resource_type_2,resource_type_3,capacity_mw_resource_1,capacity_mw_resource_2,capacity_mw_resource_3
967,993,active,4/30/2013,2013.0,NaN,NaN,NaN,CAISO,SUNSHINE VALLEY SOLAR 1,NaN,VEA,nye,NaN,NaN,NV,CAISO,Full Capacity,Valley Substation 138kV,12/31/2016,2016.0,Executed,IA Executed,Solar+Battery,Solar,Battery,NaN,-50.00,40.0,NaN
1050,1076,active,4/30/2014,2014.0,NaN,NaN,NaN,CAISO,WILLOW SPRINGS SOLAR 3,NaN,SCE,kern,NaN,NaN,CA,CAISO,Partial Capacity,Whirlwind Substation 230 kV,6/30/2017,2017.0,Executed,IA Executed,Solar+Battery,Solar,Battery,NaN,-99.00,50.0,NaN
6654,L002,active,5/11/2004,2004.0,NaN,NaN,NaN,MISO,NaN,NaN,Northern States Power (Xcel Energy),scott,NaN,NaN,MN,MISO,ERIS,Unknown,10/30/2015,2015.0,NaN,IA Executed,NaN,NaN,NaN,NaN,-28.00,NaN,NaN
6655,L003,active,5/24/2004,2004.0,NaN,NaN,NaN,MISO,NaN,NaN,"Michigan Electric Transmission Company, LLC",lenawee,NaN,NaN,MI,MISO,ERIS,Unknown,NaN,NaN,NaN,IA Executed,NaN,NaN,NaN,NaN,-10.00,NaN,NaN
6656,L004,active,7/22/2004,2004.0,NaN,NaN,NaN,MISO,NaN,NaN,FirstEnergy Solutions Corp.,williams,NaN,NaN,OH,MISO,ERIS,Unknown,4/1/2009,2009.0,NaN,IA Executed,NaN,NaN,NaN,NaN,-22.00,NaN,NaN
6657,L006,active,9/27/2004,2004.0,NaN,NaN,NaN,MISO,NaN,NaN,"Michigan Electric Transmission Company, LLC",otsego,NaN,NaN,MI,MISO,ERIS,Unknown,6/1/2005,2005.0,NaN,IA Executed,NaN,NaN,NaN,NaN,-5.00,NaN,NaN
6658,L008,active,11/15/2004,2004.0,NaN,NaN,NaN,MISO,NaN,NaN,FirstEnergy Solutions Corp.,delaware,NaN,NaN,OH,MISO,ERIS,Unknown,10/1/2004,2004.0,NaN,IA Executed,NaN,NaN,NaN,NaN,-4.00,NaN,NaN
6659,L009,active,1/14/2005,2005.0,NaN,NaN,NaN,MISO,NaN,NaN,ITC Midwest,jackson,NaN,NaN,MN,MISO,ERIS,Unknown,1/1/2005,2005.0,NaN,IA Executed,NaN,NaN,NaN,NaN,-11.00,NaN,NaN
6660,L010,active,4/1/2005,2005.0,NaN,NaN,NaN,MISO,NaN,NaN,"Michigan Electric Transmission Company, LLC",NaN,NaN,NaN,MI,MISO,ERIS,Unknown,4/1/2005,2005.0,NaN,IA Executed,NaN,NaN,NaN,NaN,-8.00,NaN,NaN
15811,18INR0064,active,4/5/2017,2017.0,10/16/2018,NaN,NaN,ERCOT,Silver Star repower,"Silver Star I Power Partners, LLC",NaN,eastland,NaN,NaN,TX,ERCOT,NaN,1642 Flat Creek 138kV,11/17/2021,2021.0,"SS Completed, FIS Completed, IA",IA Executed,Wind,Wind,NaN,NaN,-7.20,NaN,NaN


In [24]:
import dbcp.transform.lbnl_iso_queue_2021 as lbnl

In [26]:
active['project_id'] = range(active.shape[0])

/tmp/ipykernel_42/1913709520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active['project_id'] = range(active.shape[0])


In [37]:
transformed = lbnl.active_iso_queue_projects(active)

/app/dbcp/transform/helpers.py:126: FutureWarning: The default value of regex will change from True to False in a future version.
  date_strings = dates.loc[~is_numeric_string]


In [28]:
active['state'].value_counts()

TX    749
VA    719
PA    602
CA    585
OH    392
NY    381
IN    343
IL    316
AZ    268
FL    228
KY    190
NM    189
MI    162
NJ    159
OR    145
NV    141
OK    140
MA    129
WA    117
NC    115
MD    115
KS    114
NE    110
WI    106
MO    103
GA     96
LA     96
CO     91
WV     81
AR     77
ND     74
MN     73
IA     72
MT     67
ME     67
MS     65
SD     63
AL     61
UT     54
SC     51
ID     49
DE     48
RI     46
TN     46
CT     46
WY     32
NH     17
MX      7
VT      4
Name: state, dtype: int64

In [38]:
active.shape, transformed.shape

((8133, 30), (7642, 35))

In [39]:
transformed[[col for col in transformed.columns if 'date' in col]].dtypes

queue_date_raw                      object
interconnection_date_raw            object
date_withdrawn_raw                  object
date_operational_raw                object
date_proposed_raw                   object
queue_date                  datetime64[ns]
interconnection_date        datetime64[ns]
date_withdrawn              datetime64[ns]
date_operational            datetime64[ns]
date_proposed               datetime64[ns]
dtype: object

In [40]:
# some dates now only have a year, like "2020". Need to add this possibility to the parse function.
transformed[[col for col in transformed.columns if 'date' in col]].sample(15)

,queue_date_raw,interconnection_date_raw,date_withdrawn_raw,date_operational_raw,date_proposed_raw,queue_date,interconnection_date,date_withdrawn,date_operational,date_proposed
project_id,,,,,,,,,,
2512,9/19/2019,NaN,NaN,NaN,5/1/2022,2019-09-19,NaT,NaT,NaT,2022-05-01
5322,2018,NaN,NaN,NaN,12/31/2021,2018-01-01,NaT,NaT,NaT,2021-12-31
841,12/14/2018,NaN,NaN,NaN,11/8/2022,2018-12-14,NaT,NaT,NaT,2022-11-08
3069,9/22/2020,NaN,NaN,NaN,12/31/2022,2020-09-22,NaT,NaT,NaT,2022-12-31
4870,12/14/2020,NaN,NaN,NaN,11/1/2023,2020-12-14,NaT,NaT,NaT,2023-11-01
2367,3/28/2019,NaN,NaN,NaN,6/30/2022,2019-03-28,NaT,NaT,NaT,2022-06-30
1643,7/22/2021,NaN,NaN,NaN,10/31/2024,2021-07-22,NaT,NaT,NaT,2024-10-31
6961,4/15/2021,NaN,NaN,NaN,1/6/2024,2021-04-15,NaT,NaT,NaT,2024-01-06
4109,8/30/2021,NaN,NaN,NaN,11/1/2023,2021-08-30,NaT,NaT,NaT,2023-11-01


In [36]:
pd.to_numeric(active.loc[active['queue_date'].str.len() == 4, 'queue_date']).describe()

count     555.000000
mean     2018.473874
std         1.924744
min      2010.000000
25%      2017.000000
50%      2018.000000
75%      2020.000000
max      2021.000000
Name: queue_date, dtype: float64

In [42]:
active.query('county_1 == "skamania"')

,queue_id,queue_status,queue_date,queue_year,interconnection_date,date_withdrawn,date_operational,entity,project_name,developer,utility,county_1,county_2,county_3,state,region,interconnection_service_type,point_of_interconnection,date_proposed,year_proposed,interconnection_status_raw,interconnection_status_lbnl,resource_type_lbnl,resource_type_1,resource_type_2,resource_type_3,capacity_mw_resource_1,capacity_mw_resource_2,capacity_mw_resource_3,project_id
18608,G108,active,37432,2002.0,NaN,NaN,NaN,BPA,Whistling Ridge,"Whistling Ridge Energy, LLC",BPA,skamania,NaN,NaN,OR,West (non-ISO),ERIS,Midway-North Bonneville 230kV Transmission line,40178,2009.0,STUDY COMPLETED,IA Executed,Wind,Wind,NaN,NaN,70.0,NaN,NaN,6567


In [43]:
# new type: "HYBRID"
active['resource_type_1'].value_counts()

Solar             5173
Battery           1528
Wind               747
Gas                319
Offshore Wind       93
Other               92
Hydro               53
Geothermal          34
HYBRID              24
Nuclear             19
Other Storage       13
Coal                 7
CSP                  2
Pumped Storage       1
Name: resource_type_1, dtype: int64

In [44]:
active.query('resource_type_1 == "HYBRID"')

,queue_id,queue_status,queue_date,queue_year,interconnection_date,date_withdrawn,date_operational,entity,project_name,developer,utility,county_1,county_2,county_3,state,region,interconnection_service_type,point_of_interconnection,date_proposed,year_proposed,interconnection_status_raw,interconnection_status_lbnl,resource_type_lbnl,resource_type_1,resource_type_2,resource_type_3,capacity_mw_resource_1,capacity_mw_resource_2,capacity_mw_resource_3,project_id
15069,GEN-2017-208,active,2017,2017.0,NaN,NaN,NaN,SPP,NaN,NaN,WERE,mcpherson,NaN,NaN,KS,SPP,ER,Benton 138kV Substation,12/1/2021,2021.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,150.0,NaN,NaN,5041
15113,GEN-2021-064,active,2021,2021.0,NaN,NaN,NaN,SPP,NaN,NaN,NaN,caddo,NaN,NaN,OK,SPP,ER/NR,Carnegie South 138kV,2/15/2025,2025.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,100.0,NaN,NaN,5070
15141,GEN-2017-213,active,2017,2017.0,NaN,NaN,NaN,SPP,NaN,NaN,AEPW,grant,kay,NaN,OK,SPP,ER,Clarksville 345kV Substation,12/1/2021,2021.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,300.0,NaN,NaN,5085
15155,GEN-2021-063,active,2021,2021.0,NaN,NaN,NaN,SPP,NaN,NaN,NaN,mccurtain,NaN,NaN,OK,SPP,ER/NR,Craig JCT 138kV,2/15/2025,2025.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,200.0,NaN,NaN,5094
15159,GEN-2021-085,active,2021,2021.0,NaN,NaN,NaN,SPP,NaN,NaN,NaN,ford,NaN,NaN,KS,SPP,ER/NR,Crooked Creek - Fort Dodge 115kV,12/15/2027,2027.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,200.0,NaN,NaN,5097
15180,GEN-2017-205,active,2017,2017.0,NaN,NaN,NaN,SPP,NaN,NaN,WERE,mcphearson,NaN,NaN,KS,SPP,ER,East McPherson 230kV,12/1/2020,2020.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,310.0,NaN,NaN,5114
15198,GEN-2021-078,active,2021,2021.0,NaN,NaN,NaN,SPP,NaN,NaN,NaN,finney,NaN,NaN,KS,SPP,ER/NR,EPC - 115kV,10/15/2027,2027.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,160.0,NaN,NaN,5124
15282,GEN-2020-013,active,2020,2020.0,NaN,NaN,NaN,SPP,NaN,NaN,NPPD,orleans,NaN,NaN,NE,SPP,ER/NR,Holdrege Substation 115kV,10/15/2026,2026.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,215.0,NaN,NaN,5193
15323,GEN-2017-209,active,2017,2017.0,NaN,NaN,NaN,SPP,NaN,NaN,KCPL,mcpherson,NaN,NaN,KS,SPP,ER,LaCygne - Neosho 345kV,12/1/2021,2021.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,300.0,NaN,NaN,5222
15338,GEN-2020-009,active,2020,2020.0,NaN,NaN,NaN,SPP,NaN,NaN,AEP,faxon,NaN,NaN,OK,SPP,ER/NR,Lawton to Oklaunion 345kV,10/15/2026,2026.0,DISIS STAGE,In Progress,HYBRID,HYBRID,NaN,NaN,300.0,NaN,NaN,5230


In [50]:
active.query('entity == "SPP"')[['resource_type_1', 'resource_type_2']].value_counts(dropna=False)

resource_type_1  resource_type_2
Wind             NaN                189
Solar            NaN                184
Battery          NaN                 80
Solar            Battery             37
HYBRID           NaN                 24
Gas              NaN                 22
Other            NaN                 16
Wind             Battery              3
dtype: int64

In [47]:
active['resource_type_3'].value_counts()

Battery    10
Solar       1
Name: resource_type_3, dtype: int64

In [51]:
active['resource_type_lbnl'].value_counts()

Solar                        3723
Battery                      1526
Solar+Battery                1418
Wind                          721
Gas                           304
Offshore Wind                  91
Other                          91
Hydro                          47
Geothermal                     33
Wind+Battery                   25
HYBRID                         24
Nuclear                        19
Solar+Wind                     18
Gas+Battery                    13
Other Storage                  12
Solar+Wind+Battery              9
Coal                            7
Hydro+Battery                   6
Solar+Storage                   5
Battery+Other                   2
CSP                             2
Offshore Wind+Battery           2
Gas+Solar+Battery               1
Other Storage+Battery           1
Wind+Storage                    1
Gas+Solar                       1
Pumped Storage+Wind+Solar       1
Geothermal+Solar                1
Other+Battery                   1
Name: resource

In [62]:
raw_df = extract(update_path)

In [71]:
transformed_dfs = lbnl.transform(raw_df)

In [72]:
transformed_dfs.keys()

dict_keys(['iso_projects', 'iso_locations', 'iso_resource_capacity'])

In [73]:
proj = transformed_dfs['iso_projects']
loc = transformed_dfs['iso_locations']
res = transformed_dfs['iso_resource_capacity']

In [74]:
proj.head()

,region,index,queue_id,queue_status,queue_date_raw,queue_year,interconnection_date_raw,entity,project_name,developer,utility,interconnection_service_type,point_of_interconnection,date_proposed_raw,year_proposed,interconnection_status_raw,interconnection_status_lbnl,resource_type_lbnl,queue_date,interconnection_date,date_proposed
project_id,,,,,,,,,,,,,,,,,,,,,
0,CAISO,30,32,active,5/24/2004,2004.0,NaN,CAISO,TULE WIND,NaN,SDGE,Partial Capacity,Boulevard East Substation 138kV,9/1/2007,2007.0,Executed,IA Executed,Wind,2004-05-24,NaT,2007-09-01
1,CAISO,68,72,active,6/21/2005,2005.0,NaN,CAISO,LAKE ELSINORE ADVANCED PUMPED STORAGE PROJECT,NaN,SDGE,Full Capacity,Proposed Lee Lake Substation 500kV,12/31/2008,2008.0,Executed,IA Executed,Other Storage,2005-06-21,NaT,2008-12-31
2,CAISO,131,138,active,10/23/2006,2006.0,NaN,CAISO,SANDSTORM WIND POWER,NaN,SCE,Partial Capacity,Devers-Vista #1 230kV,12/31/2008,2008.0,Executed,IA Executed,Wind+Battery,2006-10-23,NaT,2008-12-31
3,CAISO,139,146,active,11/16/2006,2006.0,NaN,CAISO,DESERT SUNLIGHT PV I,NaN,SCE,Partial Capacity,Red Bluff Substation 230kV,12/1/2008,2008.0,Executed,IA Executed,Solar+Battery,2006-11-16,NaT,2008-12-01
4,CAISO,140,147,active,11/16/2006,2006.0,NaN,CAISO,DESERT SUNLIGHT PV II,NaN,SCE,Partial Capacity,Red Bluff Substation 230kV,2/1/2010,2010.0,Executed,IA Executed,Solar+Battery,2006-11-16,NaT,2010-02-01


In [75]:
loc.head()

,project_id,raw_county_name,raw_state_name,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
0,0,san diego,CA,06,06073,san diego,county,san diego
1,1,riverside,CA,06,06065,riverside,county,riverside
2,2,riverside,CA,06,06065,riverside,county,riverside
3,3,riverside,CA,06,06065,riverside,county,riverside
4,4,riverside,CA,06,06065,riverside,county,riverside


In [81]:
loc[loc['county_id_fips'].isna() & loc['raw_county_name'].notna() & loc['raw_state_name'].notna()]

,project_id,raw_county_name,raw_state_name,state_id_fips,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
181,185,tecate,MX,<NA>,<NA>,,,
182,186,baja california,MX,<NA>,<NA>,,,
185,189,sonora,MX,<NA>,<NA>,,,
252,258,municipality of tecate,MX,<NA>,<NA>,,,
553,572,baja california,MX,<NA>,<NA>,,,
576,596,tecate baja california mexico,CA,06,<NA>,CA,,
581,601,baja california,MX,<NA>,<NA>,,,
1350,1418,oliver,MN,27,<NA>,MN,,
1796,1880,unknown,OH,39,<NA>,OH,,
1848,1938,king,VA,51,<NA>,King,,


In [77]:
res.head()

,project_id,resource,capacity_mw,resource_clean
0,0,Wind,63.0,Onshore Wind
1,1,Other Storage,500.0,Other Storage
2,2,Battery,90.0,Battery Storage
3,2,Wind,60.0,Onshore Wind
4,3,Solar,NaN,Solar


In [78]:
res.groupby('resource_clean')['capacity_mw'].sum().sort_values(ascending=False)

resource_clean
Solar              633390.633
Battery Storage    327431.350
Onshore Wind       161917.280
Offshore Wind       72414.300
Natural Gas         70022.900
Unknown              7983.600
Other Storage        6630.000
Nuclear              4714.300
Other                4265.550
Hydro                4123.760
Geothermal           2168.100
Pumped Storage        870.000
Coal                  769.000
CSP                   260.000
Name: capacity_mw, dtype: float64

In [79]:
proj.shape, loc.shape, res.shape

((7642, 21), (7561, 8), (9079, 4))